# 11_패딩(Padding)
자연어 처리를 하다 보면 각 문장 혹은 문서의 길이가 서로 다른 경우가 있다. 기계는 길이가 동일한 문서들에 대해 하나의 행렬로 간주하고, 한꺼번에 묶어 처리(병렬연산)한다. 이는 연산의 속도와 효율성을 높일 수 있는 방법이기에, 이를 위해 여러 문장의 길이를 임의로 같게 맞추어 주는 작업이 필요하다. 실습을 통해 이해해보자.

## 1. Numpy로 패딩하기

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

정수 인코딩에서 수행했던 실습을 반복해보도록 한다. 아래와 같은 텍스트 데이터를 생성한다.

In [2]:
sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

정수 인코딩을 수행한다. 

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts() 안에 코퍼스를 입력하면 빈도수를 기준으로 단어 집합을 생성

이에 따라 텍스트의 모든 단어들은 빈도수의 수에 따라 순위가 매겨지게 되고, 각 단어들에는 그 순위와 같은 정수가 맵핑된다. 이 정수들을 위의 텍스트 데이터와 치환한 것이 정수 인코딩이며, 아래의 출력이다.   

In [4]:
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


모든 단어가 고유한 정수로 변환된 것을 확인할 수 있다. 이제 각 데이터를 동일한 길이로 맞추어 주기 위해 위의 데이터 중 가장 길이가 긴 문장의 길이를 계산한다.

In [5]:
max_len = max(len(item) for item in encoded)
print(max_len)

7


가장 길이가 긴 문장의 길이는 7이다. 이제 모든 문장의 길이를 7로 맞춰준다. 이 때는 가상의 단어 'PAD'를 사용한다. 'PAD'라는 단어가 있다고 가정하고, 이 단어는 0번 단어라고 정의한다. 이제 길이가 7보다 짧은 문장에는 숫자 0을 채워서 전부 길이 7로 맞춰준다. 

In [6]:
for item in encoded: # 각 문장에 대해서
    while len(item) < max_len: # max_len보다 길이가 짧을 경우
        item.append(0)
        
padded_np = np.array(encoded)
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

길이가 7보다 짧은 문장에는 모자란 만큼 숫자 0이 붙어 길이가 7로 균일하게 맞춰진 것을 볼 수 있다. 기계는 이제 이러한 데이터들을 하나의 행렬로 보고, 병렬 처리를 할 수 있게 된다. 또한 0번 단어는 아무런 의미도 없는 단어이므로 자연어 처리 과정에서 기계는 이를 무시하게 된다. 
<br>

이와 같이 데이터에 특정값을 채워 데이터의 크기(shape)을 조정하는 것을 패딩(padding)이라고 한다. 숫자 0이 특정값일 경우, 이를 제로 패딩(zero padding)이라 한다. 

## 2. 케라스 전처리 도구로 패딩하기

케라스에서는 위와 같은 패딩을 위한 도구인 pad_sequences를 ㅈ